In [1]:
import time
import bs4
import platform
import pyperclip
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
wait = 1

In [3]:
# INSTANCE BROWSER
def openBrowser(business):
    '''
        Verifica qual o sistema operacional para que use o driver e diretorio de usuarios certo
    '''
    
    if platform.system() == 'Linux':
        driver_path = './webdriver/linux/chromedriver'
        profile_path = './profile_' + business 
    elif platform.system() == 'Windows':
        driver_path = r'C:\\Users\\Yan\\Desktop\\brisbane_bot\\webdriver\\win\\chromedriver.exe'
        profile_path = "C:\\Users\\Yan\\Desktop\\Liziane - Avon\\" + business
        print(profile_path)
    
    
    global driver
    try:
        options = webdriver.ChromeOptions()
        # open with saved informations in cache
        options.add_argument("profile")
        options.add_argument("user-data-dir=" + profile_path)
        driver = webdriver.Chrome(options=options, executable_path=driver_path)
#         driver.implicitly_wait(5) #tempo implicito de espera 1 segundo antes de cada acao
        main_url = 'https://web.whatsapp.com/'
        driver.get(main_url)
        return True
        if wait_connection(driver):
            return True, 'opened in: {}'.format(main_url)
    except Exception as error:
        print(type(error), error)
        return False
    
# PARA LIBERAR SOMENTE APOS TER ABERTO O WHATSAPP
def wait_connection(driver):
    waiting = True
    while waiting:
        try:
            if driver.find_element_by_class_name('Qk8nZ').text == 'Keep your phone connected':
                waiting = False
                return True
                break
#             elif driver.find_element_by_class_name('_13HPh').text == 'Computer not connected':
#                 waiting = False
#                 return False, 'Computer not connected'
        except KeyboardInterrupt:
            waiting = False
            return False
        except:
            time.sleep(10)
            pass

In [4]:
# ENCONTRA O CONTEUDO ESPECIFICO E CLICA EM ENCAMINHAR
def sendContent(content_type):
    '''
        Recebe como parametro um tipo de conteudo e chama a funcao responsavel por clicar em encaminhar aquele conteudo
        
        precisa implementar:
            *documento
            *localizacao
            *contato
    '''
    if content_type == 'audio':
        sendAudioFile()
        return True
    elif content_type == 'image':
        sendImage()
        return True
    elif content_type == 'video':
        sendVideo()
        return True
    elif content_type == 'text':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- TEXT]')
        return False
    elif content_type == 'document':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- DOCUMENT]')
        return False
    elif content_type == 'location':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- LOCATION]')
        return False
    elif content_type == 'contact':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- CONTACT]')
        return False
    else:
        print('\nError ___ >>> sendContent <<< ___ [function: sendContent()]\n')
        return False

def sendAudioFile():
    '''
        Encontra o arquivo de audio e passa para a funcao encaminhar
    '''
    messages = driver.find_elements_by_class_name('FTBzM')
    
    for message in messages:
        try:
            # se nao der erro eh pq a mensagem eh o audio
            message.find_element_by_class_name('_1N8Pv')
            message.find_element_by_class_name('gxf3C').click()
            return True
        except Exception as error:
    #         print(type(error), error)
            pass
    
    
    # se algo nao der certo ira retornar False
    return False

def sendVideo():
    '''
        Encontra o video e passa para a funcao encaminhar
    '''
    messages = driver.find_elements_by_class_name('FTBzM')
    for message in messages:
        try:
            message.find_element_by_class_name('_3Z-uK')
            message.find_element_by_class_name('gxf3C').click()
            return True
        except Exception as error:
    #         print(type(error), error)
            pass
    
def sendImage():
    '''
        Encontra o video e passa para a funcao encaminhar
    '''
    messages = driver.find_elements_by_class_name('vW7d1')
    for message in messages:
        try:
            message.find_element_by_tag_name('img')
            message.find_element_by_class_name('_3S8Q-').click()
            return True
        except Exception as error:
#             print(type(error), error)
            pass
    return False

In [5]:
# SELECIONA CONTATOS BUSCANDO PELO NOME
def select_specified_contact(contact):
    '''
        Recebe um contato especifico, pesquisa na caixa de buscas e seleciona
    '''
    try:
        div_select_chats = driver.find_element_by_class_name('_2fq0t')
        search_box = div_select_chats.find_element_by_class_name('_2S1VP')

        for i in range(45):
            search_box.send_keys(Keys.BACKSPACE)

        search_box.send_keys(contact)
        time.sleep(wait)

        for chat in getChats():
            chat_title = chat.text.split('\n')[0]
            if chat_title == contact and clickChat(chat):
                print('selected: {}'.format(chat_title))
                return True
        return False
    except Exception as error:
        print('\nERROR -> [select_specified_contact()]\n')
        print(type(error), error)

In [6]:
# SELECIONA UM OU MAIS CONTATOS QUE AINDA NAO ENCAMINHOU PARA ENVIAR A MENSAGEM
def select_contacts(groups=False, just_one=False):
    '''
        Recebe como parametro se deve selecionar um ou mais contatos
        Verifica se os chats sao grupo, ira selecionar 5 chats que nao sejam grupos
    '''
    selected = []
    try:
        # caso nao seja para encaminhar para os grupos
        if groups == False:
            for num, chat in enumerate(getChats()):
                time.sleep(wait/2)
                chat_title = chat.text.split('\n')[0]
                if len(selected) <=4 and not isGroup(chat) and not verifyForwarded(chat_title) and clickChat(chat):
                    print('{} selected: {}\n'.format(num, chat_title))
                    selected.append(chat_title)
                    
                    ''' Se for para selecionar apenas um ira encerrar a funcao apos selecionar um'''
                    if just_one == True:
                        return True
                elif len(selected) >=4:
                    break
            return selected
        # se for para selecionar grupos eh a mesma coisa mas nao ira verificar antes se eh grupo
        else:
            for num, chat in enumerate(getChats()):
                time.sleep(wait/2)
                chat_title = chat.text.split('\n')[0]
                if len(selected) <=4 and not verifyForwarded(chat_title) and clickChat(chat):
                    chat_title = chat.text.split('\n')[0]
                    print('{} selected: {}'.format(num, chat_title))
                    selected.append(chat_title)
                    ''' Se for para selecionar apenas um ira encerrar a funcao apos selecionar um'''
                    if just_one == True:
                        return True
                elif len(selected) >=4:
                    break
            return selected
            
    except Exception as error:
        print('\nERROR -> [select_contacts()]\n')
        print(type(error), error)

def isGroup(chat):
    '''
        Rececebe um chat e verifica se eh um grupo
    
        Quado eh grupo nao tera uma div especifica, entao retornara False
    '''
    try:
        div_contact = chat.find_element_by_class_name('_3NWy8')
        return False
    except Exception as error:
        if 'Unable to locate element' not in str(error):
            print(type(error), error)
        else:
            return True
    
def click_to_send(names_to_save):
#     try:
    driver.find_element_by_xpath('//div[@data-animate-btn="true"]').click()
    ''' Salva os nomes dos contatos para os quais ja encaminhou'''
    saveForwarded(names_to_save, 'one')
    return True
#     except Exception as error:
#         print(type(error), error)
#         return False

In [7]:
def getChats():
    '''
        Pega a lista de chats apos clicar no botao de encaminhar
    '''
    try:
        chats = driver.find_elements_by_class_name('_2wP_Y')
        assert chats != [], 'NAO ENCONTROU CHATS CONVERSAS PARA SELECIONAR'
        return chats
    except Exception as error:
        print('\nERROR -> [getChats()]\n')
        print(type(error), error)
        return False

In [8]:
# SALVA CONTATOS QUE JA INTERAGIU E VERIFICA CONTATOS QUE JA INTERAGIU
def saveForwarded(names_to_save, type_of_send):
    '''
        Salva os contatos para o qual ja encaminhou.
        ha dois tipos de encaminhamento, para uma pessoa ou para mais de uma
        se for mais de uma recebera uma lista de nomes para salvar
    '''
    if type_of_send == 'one':
        with open('forwarded.txt', 'a') as filee:
            filee.write(str(names_to_save) + '\n')
        return True
    
    if type_of_send == 'list':
        for name in names_to_save:
#             print('name')
            with open('list_forwarded.txt', 'a') as filee:
                    filee.write(str(name) + '\n')
        return True
    
def verifyForwarded(chat_name=None, get_list=False):
    '''
        Verifica se ja encaminhou para aquele contato antes
    '''
    with open('forwarded.txt', 'r') as filee:
        talked = filee.read().split('\n')
    
    if chat_name != None:
        return True if chat_name in talked else False
    elif get_list != False:
        return talked

In [9]:
# ABRE UMA CONVERSA ESPECIFICA QUE CONTEM O CONTEUDO A SER ENCAMINHADO
def openChat(chat_name):
    '''
        Abre a conversa principal onde esta o conteudo a ser encaminhado
    '''

    try:
        '''Na caixa de pesquisa digita o nome do contato'''
        time.sleep(0.5)
        search_box = driver.find_element_by_class_name('_2S1VP')
        search_box.click()

        for i in range(30):
            search_box.send_keys(Keys.BACKSPACE)

        search_box.send_keys(chat_name)
        time.sleep(wait)

        ''' Clica e Abre a Conversa'''
        chats = driver.find_elements_by_class_name('_2EXPL')
        for chat in chats:
            if chat.find_element_by_class_name('_1wjpf').get_attribute('title') == chat_name:
                chat.click()
                return True
    except Exception as error:
        if 'ElementNotInteractableException' in str(type(error)):
            pass
        else:
            print('\nERROR -> [openChat()]\n')
            print(type(error), error)
            return False

In [10]:
def reWriteName(name):
    ''' recebe um nome e retorna com a primeira letra minuscula '''
    return name.split(' ')[0].lower().capitalize()

def message(contact):
    
    contact_name = reWriteName(contact)
    
    '''
        recebe um contato e retorna a mensagem personalizada
    '''
    welcome = 'Olá '+ contact_name + '!'

    message = '''

Estamos com o presente certo para o dia das mães, garanta já seu kit e muita lucratividade.
Escreve *EU QUERO* e estaremos informando a sua executiva de vendas.

Cada kit vale 1680 Pontos do Meu Mundo Avon, temos itens para resgatar e formar uma incrível coleção.'''
    
    return welcome + message

In [29]:
def sendTextMessage(contact=None):
    '''
        Envia a mensagem escrita personalizada com o nome do contato
    '''
    
    try:
        footer = driver.find_element_by_class_name('_2tW_W')
        send_text_box = footer.find_element_by_class_name('_2S1VP')

        send_text_box.click()
        
#         pyperclip.copy(message(contact))
#         send_text_box.send_keys(Keys.CONTROL + "v")
    
        message = 'Olá ' + str(contact.split(' ')[0]) + '''.

Esse é o número da nossa empresa. Estamos realizando alguns testes com o WhatsApp. 

Incluímos você na nossa lista de mensagens e pedimos a sua gentileza de *responder as mensagens* quando receber. 

As mensagens serão bem simples, apenas para que possamos *ativar a nossa conta.* 

Pode responder da forma que achar melhor, só precisamos que responda, para que o WhatsApp *não entenda a mensagem como SPAM.* Muito obrigado!!!

BHS Recuperações.'''
        
        send_text_box.send_keys(message)
        time.sleep(1)
        send_text_box.send_keys(Keys.ENTER)
        return True
    except Exception as error:
        print('\nERROR -> [sendRemarketingMessages()]\n')
        print(type(error), error)
        return False

In [22]:
sendTextMessage('Yan')

Olá Yan.

Esse é o número da nossa empresa. Estamos realizando alguns testes com o WhatsApp. 

Incluímos você na nossa lista de mensagens e pedimos a sua gentileza de *responder as mensagens* quando receber. 

As mensagens serão bem simples, apenas para que possamos *ativar a nossa conta.* 

Pode responder da forma que achar melhor, só precisamos que responda, para que o WhatsApp *não entenda a mensagem como SPAM.* Muito obrigado!!!

BHS Recuperações.


True

In [12]:
def saveTalked(list_name):
    '''
        salva os contatos que ja foram atendidos
    '''
    for name in list_name:
        #salva os contatos com os quais ja falou/fez o antendimento
        with open('forwarded.txt', 'a') as filee:
            filee.write(name + '\n')
    return True
    
def verifyTalked(chat_name=None, get_list=False):
    '''
        Verifica se ja conversou com aquele contato antes
    '''
    with open('forwarded.txt', 'r') as filee:
        talked = filee.read().split('\n')
    
    if chat_name != None:
        return True if chat_name in talked else False
    elif get_list != False:
        return talked

In [13]:
def clickChat(chat):
    try:
        chat.click()
        return True
    except Exception as error:
        print('\nERROR -> [clickChat()]\n')
        print(type(error), error)
        return False

In [14]:
def getChat(specified_contact):
    '''
        *Pesquisa um contato
        *Pega a lista de contatos que aparecem apos a pesquisa
        *Clica e abre a conversa para um contato especifico
    '''
    search_box = driver.find_element_by_class_name('_2S1VP')
    search_box.click()
    
    for i in range(45):
        search_box.send_keys(Keys.BACKSPACE)
    
    search_box.send_keys(specified_contact)
    time.sleep(1.5)
    
    for chat_contact in driver.find_elements_by_class_name('_1wjpf'):
        if chat_contact.text == specified_contact:
            try:
                chat_contact.click()
                return True
            except Exception as error:
                print(type(error), error)
        
    print('nao encontrou')
    return False
    #se nao clicar em nenhum, ira retornar False
    

In [15]:
openBrowser('BHS')

True

In [44]:
df = pd.read_csv('../Contatos/aquecimento_de_numero.csv', dtype=str)

In [45]:
contacts = df['Name'].tolist()

In [30]:
# enviados = []
errors = []

In [48]:
for position, contact in enumerate(contacts):
#     if len(enviados) > 15:
#         print('Stopped')
#         break
    try:
        '''envia a mensagem escrita'''
        if contact not in enviados:
            if getChat(contact):
                time.sleep(1)
                if sendTextMessage(contact):
                    enviados.append(contact)
                else:
                    errors.append(contact)
            else:
                errors.append(contact)
    except Exception as error:
        print(type(error), error)
        pass

In [171]:

# SALVA OS CONTATOS QUE JA FORAM ENVIADOS
for count, chat in enumerate(driver.find_elements_by_class_name('_2wP_Y')):
    print(count, chat.find_element_by_class_name('_1wjpf').text)
#     enviados.append(chat.find_element_by_class_name('_1wjpf').text)

0 Andre Querencia
1 Alexandre Mello
2 Andre Ferla
3 Carlos Diogo Santin
4 Marcos Santin
5 Andrew Brasil
6 Carlos Alberto Santin
7 Ana Maria Piccinini
8 Andre BC
9 Cristian Brasil
10 Daile Rodrigues
11 Erick Piccinini
12 Marcio Santin Junior
13 Priscila Silva
14 Thais Helena
15 Adriano Gatto


In [37]:
contacts

['Adriano Gatto',
 'Alexandre Mello',
 'Ana Maria Piccinini',
 'Andre BC',
 'Andre Ferla',
 'Andre Querencia',
 'Andrew Brasil',
 'Atendimento BHS',
 'Carlos Alberto Santin',
 'Carol Pirobano',
 'Chiquinho Putzel',
 'Cristian Brasil',
 'Daile Rodrigues',
 'Dani Elton',
 'Danilo Souza',
 'Douglas Bicudo',
 'Edvan  Brasil',
 'Elizia Santin',
 'Erick Piccinini',
 'Fernanda Belle',
 'Fernando Novaes',
 'Joaquim Mansano',
 'Jordan Bürger',
 'Marcio Santin Junior',
 'Marco Antonio Belmonte',
 'Marcos Santin',
 'Mario Marcondes',
 'Nelcy Terezinha Santin',
 'Nelson Hanteii',
 'Noelle Bonetti - Assessoria LP',
 'Priscila Silva',
 'Renata Sampaio',
 'Roberlei Prudente',
 'Rodrigo Pirobano',
 'Sandro Roberto',
 'Thais Helena',
 'Yan']

In [112]:
#abre outro csv como rererencia para pegar as colunas
df2 = pd.read_csv('files\ENVIO 1\contatos_panela_revendedores.csv')

In [115]:
contacts_df = pd.DataFrame(columns= list(df2.columns))

In [117]:
#SALVA OS CONTATOS NO NOVO DATAFRAME
for contact in range(len(df)):
    contacts_df = contacts_df.append({'Name': df.loc[contact, 'NOME REVENDEDOR'], 'Phone 1 - Value': df.loc[contact, 'TELEF. CELULAR']}, ignore_index=True)

In [118]:
# EXPORTA EM CSV
contacts_df.to_csv('revenda_with_objetivo.csv', encoding='utf-8')